In [136]:
'''
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

def convwithoutpadd(image, kernel):
    
    (img_height, img_width)=img.shape[:2]
    (kernel_height, kernel_width)=kernel.shape[:2]
    
    pad = (kernel_width - 1) / 2
    image = cv.copyMakeBorder(image, pad, pad, pad, pad, cv.BORDER_REPLICATE)
    output = np.zeros((img_height, img_width), dtype="float32")
    
    for y in np.arange(pad, img_height + pad):
            
        for x in np.arange(pad, img_width + pad):
        # extract the ROI of the image by extracting the
        # *center* region of the current (x, y)-coordinates
        # dimensions
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
 
        # perform the actual convolution by taking the
        # element-wise multiplicate between the ROI and
        # the kernel, then summing the matrix
            k = (roi * kernel).sum()
 
        # store the convolved value in the output (x,y)-
        # coordinate of the output image
            output[y - pad, x - pad] = k
    
    
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype("uint8")
    return output

img=cv.imread('messi.jpg')

kernel=np.array([[-1,-1,-1],
                 [-1,8,1],
                 [-1,-1,-1]])


b,g,r=cv.split(img)
img0=cv.merge((r,g,b))
print(int(convwithoutpadd(img, kernel)))
out_=convwithoutpadd(img, kernel)

plt.subplot(121),plt.imshow(img0),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(out),plt.title('Filtered')
plt.xticks([]), plt.yticks([])
plt.show()
'''
# import the necessary packages
from skimage.exposure import rescale_intensity
import numpy as np
from math import ceil
from math import floor
import cv2


'''def convolve(image, kernel):
	# grab the spatial dimensions of the image, along with
	# the spatial dimensions of the kernel
	(iH, iW) = image.shape[:2]
	(kH, kW) = kernel.shape[:2]
 
	# allocate memory for the output image, taking care to
	# "pad" the borders of the input image so the spatial
	# size (i.e., width and height) are not reduced
	pad = (kW - 1) / 2
	image = cv2.copyMakeBorder(image, pad, pad, pad, pad,
		cv2.BORDER_REPLICATE)
	output = np.zeros((iH, iW), dtype="float32")
	# loop over the input image, "sliding" the kernel across
	# each (x, y)-coordinate from left-to-right and top to
	# bottom
	for y in np.arange(pad, iH + pad):
		for x in np.arange(pad, iW + pad):
			# extract the ROI of the image by extracting the
			# *center* region of the current (x, y)-coordinates
			# dimensions
			roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
 
			# perform the actual convolution by taking the
			# element-wise multiplicate between the ROI and
			# the kernel, then summing the matrix
			k = (roi * kernel).sum()
 
			# store the convolved value in the output (x,y)-
			# coordinate of the output image
			output[y - pad, x - pad] = k
	# rescale the output image to be in the range [0, 255]
	output = rescale_intensity(output, in_range=(0, 255))
	output = (output * 255).astype("uint8")
 
	# return the output image
	return output
'''
def convolve(image, kernel, pad, stride):
	# grab the spatial dimensions of the image, along with
	# the spatial dimensions of the kernel
	(iH, iW) = image.shape[:2]
	(kH, kW) = kernel.shape[:2]
 
	# allocate memory for the output image, taking care to
	# "pad" the borders of the input image so the spatial
	# size (i.e., width and height) are not reduced

	image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_CONSTANT, pad)
	(a,b)=(int(((iH+2*pad-kH)/stride)+1), int(((iW+2*pad-kW)/stride)+1))
	output = np.zeros((a,b), dtype="float32")
    # loop over the input image, "sliding" the kernel across
	# each (x, y)-coordinate from left-to-right and top to
	# bottom
	for x in np.arange(pad, iH + pad, stride):
		if(x>(iH-kH)):
			break
		for y in np.arange(pad, iW+pad, stride):
			# extract the ROI of the image by extracting the
			# *center* region of the current (x, y)-coordinates
			# dimensions
			if(y>(iW-kW)):
				break
			roi = image[y -pad : y + kW, x - pad : x + kH]
#			for i in range(y-pad, y+pad+1, stride):
#				roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
#				for j in range(x-pad, x+pad+1, stride):
#					roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
			# perform the actual convolution by taking the
			# element-wise multiplicate between the ROI and
			# the kernel, then summing the matrix
			#print(roi.shape[:2], x, y, iH-kH, iW-kW)
			#print(image[y-pad:y-pad+kW])
			print(x, y, iH-kH, iW-kW, roi.shape[:2])
			k = (roi * kernel).sum()
			# store the convolved value in the output (x,y)-
			# coordinate of the output image
			output[int((y - pad)/stride), int((x - pad)/stride)] = k
            # rescale the output image to be in the range [0, 255]
	output = rescale_intensity(output, in_range=(0, 255))
	output = (output * 255).astype("uint8")
 
	# return the output image
	return output

# construct average blurring kernels used to smooth an image
smallBlur = np.ones((7, 7), dtype="float") * (1.0 / (7 * 7))
largeBlur = np.ones((21, 21), dtype="float") * (1.0 / (21 * 21))
 
# construct a sharpening filter
sharpen = np.array((
	[0, -1, 0],
	[-1, 5, -1],
	[0, -1, 0]), dtype="int")
# construct the Laplacian kernel used to detect edge-like
# regions of an image
laplacian = np.array((
	[0, 1, 0],
	[1, -4, 1],
	[0, 1, 0]), dtype="int")
 
# construct the Sobel x-axis kernel
sobelX = np.array((
	[-1, 0, 1],
	[-2, 0, 2],
	[-1, 0, 1]), dtype="int")
 
# construct the Sobel y-axis kernel
sobelY = np.array((
	[-1, -2, -1],
	[0, 0, 0],
	[1, 2, 1]), dtype="int")
# construct the kernel bank, a list of kernels we're going
# to apply using both our custom `convole` function and
# OpenCV's `filter2D` function
kernelBank = (
	("small_blur", smallBlur),
	'''("large_blur", largeBlur),
	("sharpen", sharpen),
	("laplacian", laplacian),
	("sobel_x", sobelX),
	("sobel_y", sobelY)''')

# load the input image and convert it to grayscale
image = cv2.imread('gay.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# loop over the kernels
for (kernelName, kernel) in kernelBank:
	# apply the kernel to the grayscale image using both
	# our custom `convole` function and OpenCV's `filter2D`
	# function
	print("[INFO] applying {} kernel".format(kernelName))
	convoleOutput = convolve(gray, kernel, 0, 2)
	opencvOutput = cv2.filter2D(gray, -1, kernel)
 
	# show the output images
	cv2.imshow("original", gray)
	cv2.imshow("{} - convole".format(kernelName), convoleOutput)
	cv2.imshow("{} - opencv".format(kernelName), opencvOutput)
	cv2.waitKey(0)
	cv2.destroyAllWindows()

[INFO] applying small_blur kernel
0 0 200 236 (7, 7)
0 2 200 236 (7, 7)
0 4 200 236 (7, 7)
0 6 200 236 (7, 7)
0 8 200 236 (7, 7)
0 10 200 236 (7, 7)
0 12 200 236 (7, 7)
0 14 200 236 (7, 7)
0 16 200 236 (7, 7)
0 18 200 236 (7, 7)
0 20 200 236 (7, 7)
0 22 200 236 (7, 7)
0 24 200 236 (7, 7)
0 26 200 236 (7, 7)
0 28 200 236 (7, 7)
0 30 200 236 (7, 7)
0 32 200 236 (7, 7)
0 34 200 236 (7, 7)
0 36 200 236 (7, 7)
0 38 200 236 (7, 7)
0 40 200 236 (7, 7)
0 42 200 236 (7, 7)
0 44 200 236 (7, 7)
0 46 200 236 (7, 7)
0 48 200 236 (7, 7)
0 50 200 236 (7, 7)
0 52 200 236 (7, 7)
0 54 200 236 (7, 7)
0 56 200 236 (7, 7)
0 58 200 236 (7, 7)
0 60 200 236 (7, 7)
0 62 200 236 (7, 7)
0 64 200 236 (7, 7)
0 66 200 236 (7, 7)
0 68 200 236 (7, 7)
0 70 200 236 (7, 7)
0 72 200 236 (7, 7)
0 74 200 236 (7, 7)
0 76 200 236 (7, 7)
0 78 200 236 (7, 7)
0 80 200 236 (7, 7)
0 82 200 236 (7, 7)
0 84 200 236 (7, 7)
0 86 200 236 (7, 7)
0 88 200 236 (7, 7)
0 90 200 236 (7, 7)
0 92 200 236 (7, 7)
0 94 200 236 (7, 7)
0 96 200 23

ValueError: operands could not be broadcast together with shapes (5,7) (7,7) 